In [253]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import pyarrow as pa

In [254]:
# Hyper-Parameters
PERTINENT_YEAR = 1950
MIN_NBCOUNT_TO_BE_PERTINENT = 4
WEIGTH_COMPOSITION_VS_WEATHER = 1
NUMBER_OF_WINES_TO_RECOMMEND = 10

# Read data and clean data

In [255]:
test= pq.read_table('./data/wine_xwine_vivino.parquet').to_pandas()
print(test.shape)

#save to csv
test[['WineID', 'Vintage', 'Review']].to_csv('./data/test_nb_review.csv', index=False)

(10859, 16)


In [256]:
wine_ratings = pq.read_table('./data/all_wine_xwine_vivino.parquet').to_pandas()
print(wine_ratings.shape)
wine_ratings.head()

(1015470, 14)


,WineID,Vintage,RegionID,MinRating,MaxRating,AverageRating,CountRating,WineName,Type,Elaborate,ABV,Body,Acidity,Review
0,100001,1988,1001,3.0,5.0,4.000000,2,Espumante Moscatel,Sparkling,Varietal/100%,7.5,Medium-bodied,High,None
1,100001,1999,1001,4.0,5.0,4.500000,2,Espumante Moscatel,Sparkling,Varietal/100%,7.5,Medium-bodied,High,None
2,100001,2007,1001,2.5,2.5,2.500000,1,Espumante Moscatel,Sparkling,Varietal/100%,7.5,Medium-bodied,High,None
3,100001,2008,1001,1.0,5.0,3.630952,42,Espumante Moscatel,Sparkling,Varietal/100%,7.5,Medium-bodied,High,None
4,100001,2009,1001,4.0,4.0,4.000000,1,Espumante Moscatel,Sparkling,Varietal/100%,7.5,Medium-bodied,High,None


In [257]:
#cast Vintage to int
wine_ratings['Vintage'] = wine_ratings['Vintage'].astype(int)
print(wine_ratings.columns)

Index(['WineID', 'Vintage', 'RegionID', 'MinRating', 'MaxRating',
       'AverageRating', 'CountRating', 'WineName', 'Type', 'Elaborate', 'ABV',
       'Body', 'Acidity', 'Review'],
      dtype='object')


In [258]:
#keep only where CountRating >= 4 and Vintage >= 1960
pertinent_wine_ratings = wine_ratings[wine_ratings['CountRating'] >= MIN_NBCOUNT_TO_BE_PERTINENT]
pertinent_wine_ratings = pertinent_wine_ratings[pertinent_wine_ratings['Vintage'] >= PERTINENT_YEAR]
print(pertinent_wine_ratings.shape)

(591099, 14)


# Add weather features to the table

In [259]:
# Add weather data to the dataframe by joining with ../weather_data/agg_monthly.parquet
weather_data = pd.read_parquet('./data/agg_quarterly.parquet') 

In [260]:
weather_data

,RegionID,year,quarter,avg_temperature,max_temperature,min_temperature,avg_sunshine_duration,max_sunshine_duration,min_sunshine_duration,avg_precipitation,avg_rain,avg_snowfall,count_record,avg_humidity,avg_wind_speed,avg_soil_temperature,avg_soil_moisture
0,1000,1949,1,20.630000,23.5,15.0,35311.982444,46592.96,0.00,4.676667,4.676667,0.000000,90,83.233688,7.820222,20.103980,0.439936
1,1000,1949,2,14.976923,21.9,7.0,26595.554066,39559.54,0.00,3.249451,3.249451,0.000000,91,84.216575,7.988462,16.248718,0.474114
2,1000,1949,3,13.039130,20.4,5.3,25465.872174,40515.45,0.00,3.543478,3.543478,0.000000,92,82.600091,8.897826,13.504348,0.470678
3,1000,1949,4,17.665217,24.0,11.0,36320.411304,46976.03,0.00,3.789130,3.789130,0.000000,92,76.970562,8.602582,16.346196,0.446674
4,1000,1950,1,19.893333,23.0,14.7,37219.968778,46800.00,3439.06,4.976667,4.976667,0.000000,90,77.816204,7.754722,19.935278,0.387102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637579,3750,2021,4,7.028261,17.2,-2.2,19102.866630,38550.21,0.00,2.414130,2.160870,0.178043,92,80.713768,10.784556,11.051042,0.271669
637580,3750,2022,1,4.267778,15.7,-4.9,25932.831778,40069.73,0.00,1.154444,0.397778,0.529667,90,64.812037,13.049769,5.565926,0.323090
637581,3750,2022,2,18.161538,27.4,5.4,41802.161758,51687.00,0.00,1.867033,1.839560,0.019231,91,56.547619,10.204304,14.758791,0.195355
637582,3750,2022,3,22.445652,30.7,11.7,40677.246413,51492.33,4955.21,2.007609,2.007609,0.000000,92,60.200181,9.494293,21.640580,0.206615


In [261]:
# Take into account only the weather data from 1950
weather_data = weather_data[weather_data['year'] >= PERTINENT_YEAR]
weather_data

,RegionID,year,quarter,avg_temperature,max_temperature,min_temperature,avg_sunshine_duration,max_sunshine_duration,min_sunshine_duration,avg_precipitation,avg_rain,avg_snowfall,count_record,avg_humidity,avg_wind_speed,avg_soil_temperature,avg_soil_moisture
4,1000,1950,1,19.893333,23.0,14.7,37219.968778,46800.00,3439.06,4.976667,4.976667,0.000000,90,77.816204,7.754722,19.935278,0.387102
5,1000,1950,2,14.949451,20.3,8.2,26580.660659,39600.00,0.00,5.004396,5.003297,0.000769,91,86.824634,8.153571,16.849863,0.441002
6,1000,1950,3,12.867391,20.2,6.0,26821.605978,41051.26,0.00,5.017391,5.017391,0.000000,92,83.971920,8.438451,13.822917,0.464177
7,1000,1950,4,17.859783,24.6,12.4,36485.705543,46977.23,0.00,3.581522,3.581522,0.000000,92,75.941123,8.721558,16.793659,0.394043
8,1000,1951,1,19.905556,24.1,15.0,35139.940222,46800.00,0.00,5.901111,5.901111,0.000000,90,85.733333,7.930370,20.025278,0.465602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637579,3750,2021,4,7.028261,17.2,-2.2,19102.866630,38550.21,0.00,2.414130,2.160870,0.178043,92,80.713768,10.784556,11.051042,0.271669
637580,3750,2022,1,4.267778,15.7,-4.9,25932.831778,40069.73,0.00,1.154444,0.397778,0.529667,90,64.812037,13.049769,5.565926,0.323090
637581,3750,2022,2,18.161538,27.4,5.4,41802.161758,51687.00,0.00,1.867033,1.839560,0.019231,91,56.547619,10.204304,14.758791,0.195355
637582,3750,2022,3,22.445652,30.7,11.7,40677.246413,51492.33,4955.21,2.007609,2.007609,0.000000,92,60.200181,9.494293,21.640580,0.206615


In [262]:
# Take into account only columns: RegionID, year, avg_temperature, avg_sunshine_duration, avg_precipitation, avg_rain, avg_humidity, avg_soil_temperature, avg_soil_moisture
weather_data = weather_data[['RegionID', 'year', 'quarter', 'avg_temperature', 'avg_sunshine_duration', 'avg_precipitation', 'avg_rain', 'avg_humidity', 'avg_soil_temperature', 'avg_soil_moisture']]

In [263]:
new_col = ['RegionID', 'year', 'avg_temperature_q1', 'avg_temperature_q2', 'avg_temperature_q3', 'avg_temperature_q4', 
'avg_sunshine_duration_q1', 'avg_sunshine_duration_q2', 'avg_sunshine_duration_q3', 'avg_sunshine_duration_q4', 
'avg_precipitation_q1', 'avg_precipitation_q2', 'avg_precipitation_q3', 'avg_precipitation_q4', 
'avg_rain_q1', 'avg_rain_q2', 'avg_rain_q3', 'avg_rain_q4', 
'avg_humidity_q1', 'avg_humidity_q2', 'avg_humidity_q3', 'avg_humidity_q4', 
'avg_soil_temperature_q1', 'avg_soil_temperature_q2', 'avg_soil_temperature_q3', 'avg_soil_temperature_q4', 
'avg_soil_moisture_q1', 'avg_soil_moisture_q2', 'avg_soil_moisture_q3', 'avg_soil_moisture_q4']
len(new_col)

30

In [264]:
# Create a new dataframe with the data for each region and year, and with a group of column for each quarter
new_weather_data = pd.DataFrame(columns=new_col)
new_weather_data['RegionID'] = weather_data['RegionID'].astype(int)
new_weather_data['year'] = weather_data['year'].astype(int)
# Get the avg_temperature for each quarter
new_weather_data['avg_temperature_q1'] = weather_data['avg_temperature'].where(weather_data['quarter'] == 1)
new_weather_data['avg_temperature_q2'] = weather_data['avg_temperature'].where(weather_data['quarter'] == 2)
new_weather_data['avg_temperature_q3'] = weather_data['avg_temperature'].where(weather_data['quarter'] == 3)
new_weather_data['avg_temperature_q4'] = weather_data['avg_temperature'].where(weather_data['quarter'] == 4)
# Get the avg_sunshine_duration for each quarter
new_weather_data['avg_sunshine_duration_q1'] = weather_data['avg_sunshine_duration'].where(weather_data['quarter'] == 1)
new_weather_data['avg_sunshine_duration_q2'] = weather_data['avg_sunshine_duration'].where(weather_data['quarter'] == 2)
new_weather_data['avg_sunshine_duration_q3'] = weather_data['avg_sunshine_duration'].where(weather_data['quarter'] == 3)
new_weather_data['avg_sunshine_duration_q4'] = weather_data['avg_sunshine_duration'].where(weather_data['quarter'] == 4)
# Get the avg_precipitation for each quarter
new_weather_data['avg_precipitation_q1'] = weather_data['avg_precipitation'].where(weather_data['quarter'] == 1)
new_weather_data['avg_precipitation_q2'] = weather_data['avg_precipitation'].where(weather_data['quarter'] == 2)
new_weather_data['avg_precipitation_q3'] = weather_data['avg_precipitation'].where(weather_data['quarter'] == 3)
new_weather_data['avg_precipitation_q4'] = weather_data['avg_precipitation'].where(weather_data['quarter'] == 4)
# Get the avg_rain for each quarter
new_weather_data['avg_rain_q1'] = weather_data['avg_rain'].where(weather_data['quarter'] == 1)
new_weather_data['avg_rain_q2'] = weather_data['avg_rain'].where(weather_data['quarter'] == 2)
new_weather_data['avg_rain_q3'] = weather_data['avg_rain'].where(weather_data['quarter'] == 3)
new_weather_data['avg_rain_q4'] = weather_data['avg_rain'].where(weather_data['quarter'] == 4)
# Get the avg_humidity for each quarter
new_weather_data['avg_humidity_q1'] = weather_data['avg_humidity'].where(weather_data['quarter'] == 1)
new_weather_data['avg_humidity_q2'] = weather_data['avg_humidity'].where(weather_data['quarter'] == 2)
new_weather_data['avg_humidity_q3'] = weather_data['avg_humidity'].where(weather_data['quarter'] == 3)
new_weather_data['avg_humidity_q4'] = weather_data['avg_humidity'].where(weather_data['quarter'] == 4)
# Get the avg_soil_temperature for each quarter
new_weather_data['avg_soil_temperature_q1'] = weather_data['avg_soil_temperature'].where(weather_data['quarter'] == 1)
new_weather_data['avg_soil_temperature_q2'] = weather_data['avg_soil_temperature'].where(weather_data['quarter'] == 2)
new_weather_data['avg_soil_temperature_q3'] = weather_data['avg_soil_temperature'].where(weather_data['quarter'] == 3)
new_weather_data['avg_soil_temperature_q4'] = weather_data['avg_soil_temperature'].where(weather_data['quarter'] == 4)
# Get the avg_soil_moisture for each quarter
new_weather_data['avg_soil_moisture_q1'] = weather_data['avg_soil_moisture'].where(weather_data['quarter'] == 1)
new_weather_data['avg_soil_moisture_q2'] = weather_data['avg_soil_moisture'].where(weather_data['quarter'] == 2)
new_weather_data['avg_soil_moisture_q3'] = weather_data['avg_soil_moisture'].where(weather_data['quarter'] == 3)
new_weather_data['avg_soil_moisture_q4'] = weather_data['avg_soil_moisture'].where(weather_data['quarter'] == 4)


In [265]:
new_weather_data

,RegionID,year,avg_temperature_q1,avg_temperature_q2,avg_temperature_q3,avg_temperature_q4,avg_sunshine_duration_q1,avg_sunshine_duration_q2,avg_sunshine_duration_q3,avg_sunshine_duration_q4,...,avg_humidity_q3,avg_humidity_q4,avg_soil_temperature_q1,avg_soil_temperature_q2,avg_soil_temperature_q3,avg_soil_temperature_q4,avg_soil_moisture_q1,avg_soil_moisture_q2,avg_soil_moisture_q3,avg_soil_moisture_q4
4,1000,1950,19.893333,NaN,NaN,NaN,37219.968778,NaN,NaN,NaN,...,NaN,NaN,19.935278,NaN,NaN,NaN,0.387102,NaN,NaN,NaN
5,1000,1950,NaN,14.949451,NaN,NaN,NaN,26580.660659,NaN,NaN,...,NaN,NaN,NaN,16.849863,NaN,NaN,NaN,0.441002,NaN,NaN
6,1000,1950,NaN,NaN,12.867391,NaN,NaN,NaN,26821.605978,NaN,...,83.971920,NaN,NaN,NaN,13.822917,NaN,NaN,NaN,0.464177,NaN
7,1000,1950,NaN,NaN,NaN,17.859783,NaN,NaN,NaN,36485.705543,...,NaN,75.941123,NaN,NaN,NaN,16.793659,NaN,NaN,NaN,0.394043
8,1000,1951,19.905556,NaN,NaN,NaN,35139.940222,NaN,NaN,NaN,...,NaN,NaN,20.025278,NaN,NaN,NaN,0.465602,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637579,3750,2021,NaN,NaN,NaN,7.028261,NaN,NaN,NaN,19102.866630,...,NaN,80.713768,NaN,NaN,NaN,11.051042,NaN,NaN,NaN,0.271669
637580,3750,2022,4.267778,NaN,NaN,NaN,25932.831778,NaN,NaN,NaN,...,NaN,NaN,5.565926,NaN,NaN,NaN,0.323090,NaN,NaN,NaN
637581,3750,2022,NaN,18.161538,NaN,NaN,NaN,41802.161758,NaN,NaN,...,NaN,NaN,NaN,14.758791,NaN,NaN,NaN,0.195355,NaN,NaN
637582,3750,2022,NaN,NaN,22.445652,NaN,NaN,NaN,40677.246413,NaN,...,60.200181,NaN,NaN,NaN,21.640580,NaN,NaN,NaN,0.206615,NaN


In [266]:
new_weather_data.dtypes

RegionID                      int32
year                          int32
avg_temperature_q1          float64
avg_temperature_q2          float64
avg_temperature_q3          float64
avg_temperature_q4          float64
avg_sunshine_duration_q1    float64
avg_sunshine_duration_q2    float64
avg_sunshine_duration_q3    float64
avg_sunshine_duration_q4    float64
avg_precipitation_q1        float64
avg_precipitation_q2        float64
avg_precipitation_q3        float64
avg_precipitation_q4        float64
avg_rain_q1                 float64
avg_rain_q2                 float64
avg_rain_q3                 float64
avg_rain_q4                 float64
avg_humidity_q1             float64
avg_humidity_q2             float64
avg_humidity_q3             float64
avg_humidity_q4             float64
avg_soil_temperature_q1     float64
avg_soil_temperature_q2     float64
avg_soil_temperature_q3     float64
avg_soil_temperature_q4     float64
avg_soil_moisture_q1        float64
avg_soil_moisture_q2        

In [267]:
# Group the data by RegionID and year and take the max of each column
new_weather_data = new_weather_data.groupby(['RegionID', 'year']).max().reset_index()

In [268]:
# rename the column RegionID to Region
new_weather_data.head()

,RegionID,year,avg_temperature_q1,avg_temperature_q2,avg_temperature_q3,avg_temperature_q4,avg_sunshine_duration_q1,avg_sunshine_duration_q2,avg_sunshine_duration_q3,avg_sunshine_duration_q4,...,avg_humidity_q3,avg_humidity_q4,avg_soil_temperature_q1,avg_soil_temperature_q2,avg_soil_temperature_q3,avg_soil_temperature_q4,avg_soil_moisture_q1,avg_soil_moisture_q2,avg_soil_moisture_q3,avg_soil_moisture_q4
0,1000,1950,19.893333,14.949451,12.867391,17.859783,37219.968778,26580.660659,26821.605978,36485.705543,...,83.971920,75.941123,19.935278,16.849863,13.822917,16.793659,0.387102,0.441002,0.464177,0.394043
1,1000,1951,19.905556,14.984615,13.528261,18.303261,35139.940222,29309.947033,29095.099457,33248.784348,...,81.099638,82.100996,20.025278,16.618498,14.367255,17.578487,0.465602,0.402114,0.456512,0.473211
2,1000,1952,21.251648,13.596703,13.494565,18.288043,36832.307802,24504.818462,25644.408043,39615.146304,...,84.093297,74.871830,20.884844,16.249222,14.006476,17.399909,0.416579,0.467314,0.475780,0.427901
3,1000,1953,21.611111,15.090110,13.854348,17.969565,36866.511889,26820.384396,26967.573478,33055.137826,...,84.485960,81.203804,21.258935,17.257280,14.582246,17.492618,0.401090,0.415662,0.481943,0.459199
4,1000,1954,20.932222,14.176923,13.298913,18.375000,34904.061333,23925.035385,22158.287826,37201.276196,...,83.751812,75.745924,20.642685,16.334844,14.073279,17.538179,0.456501,0.470999,0.457741,0.412033


In [269]:
print(new_weather_data.shape)

(157242, 30)


In [270]:
#Join the weather data with the pertinent_wine_ratings dataframe
pertinent_wine_ratings_with_weather = pertinent_wine_ratings.merge(new_weather_data, left_on=['RegionID', 'Vintage'], right_on=['RegionID', 'year'])
# Drop the column year
pertinent_wine_ratings_with_weather = pertinent_wine_ratings_with_weather.drop(['year'], axis=1)

print(pertinent_wine_ratings_with_weather.shape)

(589279, 42)


In [271]:
print(pertinent_wine_ratings_with_weather.columns)

Index(['WineID', 'Vintage', 'RegionID', 'MinRating', 'MaxRating',
       'AverageRating', 'CountRating', 'WineName', 'Type', 'Elaborate', 'ABV',
       'Body', 'Acidity', 'Review', 'avg_temperature_q1', 'avg_temperature_q2',
       'avg_temperature_q3', 'avg_temperature_q4', 'avg_sunshine_duration_q1',
       'avg_sunshine_duration_q2', 'avg_sunshine_duration_q3',
       'avg_sunshine_duration_q4', 'avg_precipitation_q1',
       'avg_precipitation_q2', 'avg_precipitation_q3', 'avg_precipitation_q4',
       'avg_rain_q1', 'avg_rain_q2', 'avg_rain_q3', 'avg_rain_q4',
       'avg_humidity_q1', 'avg_humidity_q2', 'avg_humidity_q3',
       'avg_humidity_q4', 'avg_soil_temperature_q1', 'avg_soil_temperature_q2',
       'avg_soil_temperature_q3', 'avg_soil_temperature_q4',
       'avg_soil_moisture_q1', 'avg_soil_moisture_q2', 'avg_soil_moisture_q3',
       'avg_soil_moisture_q4'],
      dtype='object')


# Create and train a doc2vec model on the colume 'review'

In [272]:
#drop null values in review column
pertinent_ratings_non_null = pertinent_wine_ratings.dropna(subset=['Review'])
print(pertinent_ratings_non_null.shape)

(9019, 14)


In [273]:
reviews = pertinent_ratings_non_null['Review']
unique_reviews = reviews.unique()

#convert to pandas series
unique_reviews = pd.Series(unique_reviews)
print(len(unique_reviews))

8699


In [274]:
# import nltk
# nltk.download()

In [275]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) | set(stopwords.words('french')) | set(stopwords.words('spanish'))
import string

def clean_review_for_doc2vec(review):
    #Clean the unique reviews series with lower case and remove punctuation
    review = review.lower()
    review = review.translate(str.maketrans('', '', string.punctuation))
    #Remove stop words in each review
    review = review.split()
    review = [word for word in review if not word in stop_words]
    review = ' '.join(review)
    #remove emojis in each review
    review = review.encode('ascii', 'ignore').decode('ascii')
    return review

unique_reviews_no_punctuation_no_stop_word = unique_reviews.apply(clean_review_for_doc2vec)

In [276]:
print(unique_reviews[1])
print(unique_reviews_no_punctuation_no_stop_word[1])

Qta. Manoella’s flagship cuvée; a field blend of 30+ varieties; 120+yo vines; 20m in 50% new oak. Tinta Francisca is unusually a high proportion of the blend.Deep ruby. Fantastic fragrance w. tobacco; leather & stylish fruit of black cherry & layered blackberry w. herbal notes. Developing w. loganberry & mulberry.Dry. V. rich & pure w. cherry; raspberry; loganberry; blueberry & black cherry. Generous m+ fine tannins; m+ acid; high alc; discreet oak & a huge finish. Outstanding wine w. power & elegance. 
qta manoellas flagship cuve field blend 30 varieties 120yo vines 20m 50 new oak tinta francisca unusually high proportion blenddeep ruby fantastic fragrance w tobacco leather stylish fruit black cherry layered blackberry w herbal notes developing w loganberry mulberrydry v rich pure w cherry raspberry loganberry blueberry black cherry generous fine tannins acid high alc discreet oak huge finish outstanding wine w power elegance


In [277]:
#Find the longest review in unique_reviews
max_len = 0
for x in unique_reviews_no_punctuation_no_stop_word:
    if len(x.split()) > max_len:
        max_len = len(x.split())
print(max_len)

69


In [278]:
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from nltk.tokenize import word_tokenize

In [279]:
# preproces the documents, and create TaggedDocuments
tagged_data = [TaggedDocument(words=word_tokenize(doc.lower()),
                              tags=[str(i)]) for i,
               doc in enumerate(unique_reviews_no_punctuation_no_stop_word)]

In [280]:
# train the Doc2vec model
model = Doc2Vec(vector_size=100,
                min_count=5, epochs=50)
model.build_vocab(tagged_data)
model.train(tagged_data,
            total_examples=model.corpus_count,
            epochs=model.epochs)

In [281]:
# save the model
model.save("./data/doc2vec.model")

In [282]:
# load the model
model= Doc2Vec.load("./data/doc2vec.model")

In [283]:
cleaned_reviews = reviews.apply(clean_review_for_doc2vec)
print(len(cleaned_reviews))

9019


In [284]:
# get the document vectors
document_vectors = [model.infer_vector(
    word_tokenize(doc)) for doc in cleaned_reviews]

In [285]:
#convert document vectors to dataframe
document_vectors_df = pd.DataFrame(document_vectors, columns=['doc2vec'+str(i) for i in range(len(document_vectors[0]))])
print(document_vectors_df.shape)

#find the min of each column
print(min(document_vectors_df.min()))
print(max(document_vectors_df.max()))

(9019, 100)
-3.10115909576416
2.461682081222534


In [286]:
#get the wineID and review columns from pertinent_wine_ratings
key = pertinent_ratings_non_null[['WineID', 'Vintage']]
print(key.shape)
#concatenate key and document_vectors_df vertically
wine_with_only_text_review = pd.concat([key.reset_index(drop=True),document_vectors_df.reset_index(drop=True)], axis=1)
print(wine_with_only_text_review.shape)


(9019, 2)
(9019, 102)


In [287]:
#aggregate the document vectors by wineID and vintage -> average
aggregated_doc_vector = wine_with_only_text_review.groupby(['WineID', 'Vintage']).mean().reset_index()

aggregated_doc_vector.head()

,WineID,Vintage,doc2vec0,doc2vec1,doc2vec2,doc2vec3,doc2vec4,doc2vec5,doc2vec6,doc2vec7,...,doc2vec90,doc2vec91,doc2vec92,doc2vec93,doc2vec94,doc2vec95,doc2vec96,doc2vec97,doc2vec98,doc2vec99
0,102356,2018,0.186890,-0.055911,-0.164264,-0.095103,0.208650,-0.250062,-0.041833,0.440843,...,0.267686,0.159916,0.161963,0.196642,0.267894,0.012079,-0.031804,-0.161102,-0.041043,-0.308405
1,106708,2003,0.083254,-0.000910,0.171584,-0.268555,-0.037198,-0.558989,-0.085905,0.355074,...,0.418082,0.100849,0.179757,-0.187356,0.021629,0.124092,0.102664,0.138566,-0.093847,-0.062429
2,106708,2005,0.101113,0.015283,0.174136,-0.235218,0.110419,-0.388743,0.066428,0.438469,...,0.396492,0.217753,0.140745,-0.022811,0.076054,0.182790,0.243012,-0.070726,-0.098792,0.009622
3,106791,2020,-0.030709,-0.061822,-0.038584,-0.050496,0.236945,-0.135342,0.214123,0.363055,...,0.374070,0.105146,0.171509,-0.007625,-0.144675,0.027952,0.281263,-0.045576,-0.228426,-0.123816
4,107349,2012,0.100363,-0.061188,-0.052852,-0.035965,0.076791,-0.324644,-0.199969,0.276602,...,0.265027,-0.018055,0.111635,-0.008229,0.122746,0.074876,-0.055842,-0.062555,0.018184,-0.196862


In [288]:
print(aggregated_doc_vector.shape)

(451, 102)


# Comparison models

Let's separate the data that can be calculate with a distance from the others

In [289]:
data_to_normalize = pertinent_wine_ratings_with_weather.drop(['RegionID', 'MinRating', 'MaxRating','Type','CountRating', 'Review'], axis=1)
data_to_normalize.dropna(inplace=True)
data_to_normalize.drop_duplicates(inplace=True)
print(data_to_normalize.shape) 

(580749, 36)


In [290]:
print(data_to_normalize.columns)

Index(['WineID', 'Vintage', 'AverageRating', 'WineName', 'Elaborate', 'ABV',
       'Body', 'Acidity', 'avg_temperature_q1', 'avg_temperature_q2',
       'avg_temperature_q3', 'avg_temperature_q4', 'avg_sunshine_duration_q1',
       'avg_sunshine_duration_q2', 'avg_sunshine_duration_q3',
       'avg_sunshine_duration_q4', 'avg_precipitation_q1',
       'avg_precipitation_q2', 'avg_precipitation_q3', 'avg_precipitation_q4',
       'avg_rain_q1', 'avg_rain_q2', 'avg_rain_q3', 'avg_rain_q4',
       'avg_humidity_q1', 'avg_humidity_q2', 'avg_humidity_q3',
       'avg_humidity_q4', 'avg_soil_temperature_q1', 'avg_soil_temperature_q2',
       'avg_soil_temperature_q3', 'avg_soil_temperature_q4',
       'avg_soil_moisture_q1', 'avg_soil_moisture_q2', 'avg_soil_moisture_q3',
       'avg_soil_moisture_q4'],
      dtype='object')


### Encode categorical data

In [291]:
#create the dictionary for the categorical variables

acid = data_to_normalize['Acidity'].unique()
print(acid)

body = data_to_normalize['Body'].unique()
print(body)

elaborate = data_to_normalize['Elaborate'].unique()
print(elaborate)

acid_dict = {'Low': 1, 'Medium': 2, 'High': 3}
body_dict = {'Very light-bodied': 1, 'Light-bodied': 2, 'Medium-bodied': 3, 'Full-bodied': 4, 'Very full-bodied': 5}
elaborat_dict = {'Varietal/100%':1, 'Varietal/>75%': 2, 'Assemblage/Blend' : 3,
                 'Assemblage/Meritage Red Blend':4, 'Assemblage/Meritage White Blend': 5, 
                 'Assemblage/Rhône Red Blend':6, 'Assemblage/Bordeaux Red Blend':7 , 
                 'Assemblage/Bourgogne Red Blend': 8, 'Assemblage/Bourgogne White Blend': 9, 'Assemblage/Portuguese White Blend': 10, 
                 'Assemblage/Portuguese Red Blend': 11, 'Assemblage/Port Blend': 12,  
                 'Assemblage/Provence Rosé Blend' :13, 'Assemblage/Champagne Blend': 14, 'Assemblage/Valpolicella Red Blend': 15,
                 'Assemblage/Chianti Red Blend': 16, 'Assemblage/Tuscan Red Blend': 17, 'Assemblage/Rioja Red Blend': 18, 
                 'Assemblage/Rioja White Blend' : 19, 'Assemblage/Priorat Red Blend': 20,
                 'Assemblage/Cava Blend': 21, 'Assemblage/Soave White Blend': 22
                 }

['High' 'Medium' 'Low']
['Medium-bodied' 'Full-bodied' 'Light-bodied' 'Very light-bodied'
 'Very full-bodied']
['Varietal/100%' 'Assemblage/Bordeaux Red Blend' 'Assemblage/Blend'
 'Assemblage/Portuguese Red Blend' 'Assemblage/Portuguese White Blend'
 'Varietal/>75%' 'Assemblage/Port Blend' 'Assemblage/Champagne Blend'
 'Assemblage/Rhône Red Blend' 'Assemblage/Tuscan Red Blend'
 'Assemblage/Chianti Red Blend' 'Assemblage/Valpolicella Red Blend'
 'Assemblage/Bourgogne Red Blend' 'Assemblage/Meritage Red Blend'
 'Assemblage/Provence Rosé Blend' 'Assemblage/Soave White Blend'
 'Assemblage/Rioja Red Blend' 'Assemblage/Rioja White Blend'
 'Assemblage/Cava Blend' 'Assemblage/Priorat Red Blend'
 'Assemblage/Bourgogne White Blend' 'Assemblage/Meritage White Blend']


In [292]:
## Label Encoding
data_to_normalize['Body'] = data_to_normalize['Body'].map(body_dict)
data_to_normalize['Acidity'] = data_to_normalize['Acidity'].map(acid_dict)
data_to_normalize['Elaborate'] = data_to_normalize['Elaborate'].map(elaborat_dict)

data_to_normalize.head()

,WineID,Vintage,AverageRating,WineName,Elaborate,ABV,Body,Acidity,avg_temperature_q1,avg_temperature_q2,...,avg_humidity_q3,avg_humidity_q4,avg_soil_temperature_q1,avg_soil_temperature_q2,avg_soil_temperature_q3,avg_soil_temperature_q4,avg_soil_moisture_q1,avg_soil_moisture_q2,avg_soil_moisture_q3,avg_soil_moisture_q4
0,100001,2008,3.630952,Espumante Moscatel,1,7.5,3,3,25.573626,26.036264,...,37.162138,70.893116,26.670238,26.411035,27.622464,28.655389,0.478863,0.420179,0.343988,0.375222
1,100005,2008,2.946429,Maison de Ville Cabernet-Merlot,7,11.0,4,2,25.573626,26.036264,...,37.162138,70.893116,26.670238,26.411035,27.622464,28.655389,0.478863,0.420179,0.343988,0.375222
2,100006,2008,3.378788,Reserva Cabernet Sauvignon,1,12.5,4,3,25.573626,26.036264,...,37.162138,70.893116,26.670238,26.411035,27.622464,28.655389,0.478863,0.420179,0.343988,0.375222
3,100016,2008,2.713415,Acquasantiera Tinto Seco Fino,3,11.0,3,1,25.573626,26.036264,...,37.162138,70.893116,26.670238,26.411035,27.622464,28.655389,0.478863,0.420179,0.343988,0.375222
4,100025,2008,3.500000,Espumante Método Charmat Brut,3,12.0,3,3,25.573626,26.036264,...,37.162138,70.893116,26.670238,26.411035,27.622464,28.655389,0.478863,0.420179,0.343988,0.375222


### Normalize the data

In [293]:
# The wine_dataset must contain the following columns: ['WineID','Vintage', 'WineName']. 
# The other columns are the features.
def normalize_wine_data(wine_dataset):
    ## Select Features and Target
    features = wine_dataset.drop(['WineID','Vintage', 'WineName'], axis=1)
    targets = wine_dataset[['WineID','Vintage', 'WineName']]
    ## Normalize Features
    for column in features.columns:
        features[column] = (features[column] - features[column].min()) / (features[column].max() - features[column].min())
    ## Return normalized dataset
    normalized_df = pd.concat([targets.reset_index(drop=True),features.reset_index(drop=True)], axis=1)
    return normalized_df

In [294]:
# Normalize the data
normalized_wine_data = normalize_wine_data(data_to_normalize)
print(normalized_wine_data.shape)
print(normalized_wine_data.head())

(580749, 36)
   WineID  Vintage                         WineName  AverageRating  Elaborate  \
0  100001     2008               Espumante Moscatel       0.657738   0.000000   
1  100005     2008  Maison de Ville Cabernet-Merlot       0.486607   0.285714   
2  100006     2008       Reserva Cabernet Sauvignon       0.594697   0.000000   
3  100016     2008    Acquasantiera Tinto Seco Fino       0.428354   0.095238   
4  100025     2008    Espumante Método Charmat Brut       0.625000   0.095238   

        ABV  Body  Acidity  avg_temperature_q1  avg_temperature_q2  ...  \
0  0.156250  0.50      1.0            0.895089            0.815914  ...   
1  0.229167  0.75      0.5            0.895089            0.815914  ...   
2  0.260417  0.75      1.0            0.895089            0.815914  ...   
3  0.229167  0.50      0.0            0.895089            0.815914  ...   
4  0.250000  0.50      1.0            0.895089            0.815914  ...   

   avg_humidity_q3  avg_humidity_q4  avg_soil_tem

In [295]:
normalized_wine_name_data = normalized_wine_data[['WineID', 'Vintage', 'WineName']]
normalized_wine_calculate_data = normalized_wine_data.drop(['WineID', 'Vintage', 'WineName'], axis=1)
print(normalized_wine_calculate_data.shape)

(580749, 33)


In [296]:
normalized_wine_calculate_data[['Elaborate', 'ABV', 'Body', 'Acidity']] = normalized_wine_calculate_data[['Elaborate', 'ABV', 'Body', 'Acidity']] * WEIGTH_COMPOSITION_VS_WEATHER

### Now let's add Doc2Vec vector

In [297]:
wine_text_review_vec = aggregated_doc_vector.copy()
wine_name_data = wine_text_review_vec[['WineID', 'Vintage']]
wine_vec = wine_text_review_vec.drop(['WineID', 'Vintage'], axis=1)

print(wine_vec.shape)

(451, 100)


# Now, let's compare using KD Tree

In [298]:
wine_id = 102356
vintage = 2018

In [299]:
checked_exist_df = pertinent_wine_ratings[(pertinent_wine_ratings['WineID'] == wine_id) & (pertinent_wine_ratings['Vintage'] == vintage)]

In [300]:
# Check if the wine_id and vintage are in the dataset
if len(checked_exist_df) > 0:
    reference_wine_composition_and_weather = normalized_wine_data[(normalized_wine_data['WineID'] == wine_id) & (normalized_wine_data['Vintage'] == vintage)] 
else:
    print("Wine ID and Vintage not found in the dataset")

In [301]:
check_review_df = pertinent_ratings_non_null[(pertinent_ratings_non_null['WineID'] == wine_id) & (pertinent_ratings_non_null['Vintage'] == vintage)]

In [302]:
# Check if the wine_id and vintage have at least 1 text review
have_text_review = False
if (len(check_review_df) > 0):
    print("Wine ID found wirh text review")
    have_text_review = True
    reference_wine_text_review = wine_text_review_vec[(wine_text_review_vec['WineID'] == wine_id) & (wine_text_review_vec['Vintage'] == vintage)]
else:
    print("Wine ID found with no text review")

Wine ID found wirh text review


In [303]:
input_wine_composition_and_weather = reference_wine_composition_and_weather.drop(['WineID','Vintage', 'WineName'], axis=1).to_numpy().reshape(1,-1)
print(input_wine_composition_and_weather)

if have_text_review:
    input_wine_text_review = reference_wine_text_review.drop(['WineID','Vintage'], axis=1).to_numpy().reshape(1,-1)
    print(input_wine_text_review.shape)

[[0.875      0.         0.30208333 1.         1.         0.54339241
  0.49744684 0.70945584 0.59230027 0.3504532  0.71812778 0.9411702
  0.40439135 0.14405233 0.12795747 0.00926318 0.09904919 0.1322163
  0.14119311 0.00926318 0.10036257 0.81203195 0.63924409 0.4219436
  0.78768603 0.365504   0.49014703 0.66964533 0.49825435 0.14896233
  0.1929745  0.18920456 0.23225387]]
(1, 100)


In [304]:
from sklearn.neighbors import KDTree
# Build the KD Tree
wine_composition_weather_tree = KDTree(normalized_wine_calculate_data, metric='euclidean')

if have_text_review:
    wine_text_review_tree = KDTree(wine_vec, metric='euclidean')

In [305]:
# Query the KD Tree
dist_composition_weather, ind_composition_weather = wine_composition_weather_tree.query(input_wine_composition_and_weather, k=len(normalized_wine_calculate_data))

if have_text_review:
    dist_text_review, ind_text_review = wine_text_review_tree.query(input_wine_text_review, k=len(wine_vec))

# Print the results
print(dist_composition_weather.shape)
print(ind_composition_weather)

if have_text_review:
    print(dist_text_review.shape)
    print(ind_text_review.shape)

(1, 580749)
[[ 53405  53105  52955 ...  16860  16898 550097]]
(1, 451)
(1, 451)


In [306]:
# create a dictionary with ind as key and dist as value
dict_composition_weather_wine = dict(zip(ind_composition_weather[0], dist_composition_weather[0]))
print(dict_composition_weather_wine[53405])

if have_text_review:
    dict_text_review_wine = dict(zip(ind_text_review[0], dist_text_review[0]))

0.0


In [307]:
# merge dictionary with normalized_wine_data with the value of the key is the index of the row in normalized_wine_data
# and the value is the distance
normalized_wine_name_data['distance1'] = normalized_wine_data.index.map(dict_composition_weather_wine)
print(normalized_wine_name_data.head())

# find max of distance1
max_distance1 = normalized_wine_name_data['distance1'].max()
print(max_distance1)

   WineID  Vintage                         WineName  distance1
0  100001     2008               Espumante Moscatel   1.735967
1  100005     2008  Maison de Ville Cabernet-Merlot   1.801502
2  100006     2008       Reserva Cabernet Sauvignon   1.684612
3  100016     2008    Acquasantiera Tinto Seco Fino   2.039361
4  100025     2008    Espumante Método Charmat Brut   1.737640
2.3621101489437346


C:\Users\bbbbb\AppData\Local\Temp\ipykernel_32676\3945777354.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalized_wine_name_data['distance1'] = normalized_wine_data.index.map(dict_composition_weather_wine)


In [308]:
if have_text_review:
    wine_name_data['distance2'] = wine_text_review_vec.index.map(dict_text_review_wine)
    print(wine_name_data.head())

    # find max of distance2
    max_distance2 = wine_name_data['distance2'].max()
    print(max_distance2)

   WineID  Vintage  distance2
0  102356     2018   0.000000
1  106708     2003   2.211803
2  106708     2005   2.548448
3  106791     2020   1.977834
4  107349     2012   1.665184
2.548447531676741


C:\Users\bbbbb\AppData\Local\Temp\ipykernel_32676\1343433895.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wine_name_data['distance2'] = wine_text_review_vec.index.map(dict_text_review_wine)


In [309]:
# Merge the datasets based on 'Name' and 'Age'
merged_wine_data = pd.merge(normalized_wine_name_data, wine_name_data, on=['WineID', 'Vintage'],how='left')

if have_text_review:
    # Fill null values with 0 before adding 'ScoreDay1' and 'ScoreDay2'
    merged_wine_data['distance'] = merged_wine_data['distance1'].fillna(0) + merged_wine_data['distance2'].fillna(0)

    # Drop the redundant 'ScoreDay1' and 'ScoreDay2' columns if needed
    merged_wine_data = merged_wine_data.drop(['distance1', 'distance2'], axis=1)

In [310]:
print(merged_wine_data.shape)
print(merged_wine_data.head())

(580749, 4)
   WineID  Vintage                         WineName  distance
0  100001     2008               Espumante Moscatel  1.735967
1  100005     2008  Maison de Ville Cabernet-Merlot  1.801502
2  100006     2008       Reserva Cabernet Sauvignon  1.684612
3  100016     2008    Acquasantiera Tinto Seco Fino  2.039361
4  100025     2008    Espumante Método Charmat Brut  1.737640


In [311]:
# Sort the DataFrame by the euclidean_distance column and display the 10 nearest wines
ref_wine_name = normalized_wine_data.loc[(normalized_wine_data['WineID'] == wine_id) & (normalized_wine_data['Vintage'] == vintage)]['WineName'].values[0]
print("reference wine is "+ ref_wine_name + " in year " + str(vintage))
print("The 10 most similar wines are:")

if have_text_review:
    distance_column = 'distance'
else:
    distance_column = 'distance1'

sorted_df = merged_wine_data.sort_values(by=[distance_column])

top_10 = sorted_df.head(NUMBER_OF_WINES_TO_RECOMMEND+1)

for index, row in top_10.iterrows():
    print(row['WineName'] + " in year " + str(row['Vintage']) +" with ID " + str(row['WineID']) +" with distance " + str(row[distance_column]))

reference wine is Douro Quinta da Manoella VV Tinto in year 2018
The 10 most similar wines are:
Vinhas Velhas Limited Release Touriga Nacional in year 2018 with ID 104887 with distance 0.0
Douro Quinta da Manoella VV Tinto in year 2018 with ID 102356 with distance 0.0
Douro in year 2018 with ID 101634 with distance 0.0029761904761904656
Reserva Vinhas Velhas in year 2018 with ID 101578 with distance 0.004999999999999893
Touriga Nacional Reserva in year 2018 with ID 103239 with distance 0.011904761904761862
Reserva in year 2018 with ID 103105 with distance 0.014731391274719766
Douro Reserva Red in year 2018 with ID 102018 with distance 0.015625
Guyot Tinto in year 2018 with ID 106354 with distance 0.017857142857142794
Syrah in year 2018 with ID 105635 with distance 0.017857142857142794
Altitude in year 2018 with ID 106213 with distance 0.01877891289304159
Reserva Touriga Nacional in year 2018 with ID 106216 with distance 0.01877891289304159


In [312]:
normalized_wine_data.to_parquet('./data/normalized_wine_data.parquet', engine='pyarrow', index=True)

In [313]:
aggregated_doc_vector.to_csv('./data/aggregated_doc_vector.csv', index=True)

In [314]:
pertinent_ratings_non_null.to_parquet('./data/pertinent_ratings_non_null.parquet', engine='pyarrow', index=True)

In [315]:
pertinent_wine_ratings.to_parquet('./data/pertinent_wine_ratings.parquet', engine='pyarrow', index=True)

In [247]:
# Save the KD Tree(s)
import joblib   

if have_text_review:
    joblib.dump(wine_text_review_tree, './data/wine_text_review_tree.joblib')

joblib.dump(wine_composition_weather_tree, './data/wine_composition_weather_tree.joblib')

['./data/wine_composition_weather_tree.joblib']